In [2]:
import numpy as np
import librosa

def textures(x, texture_len=5, texture_step=5):
    tx = []
    for i in range( int( (len(x)-texture_len)/texture_step) ):
        this_frame = x[int(i*texture_step) :int(i*texture_step + texture_len), :]
        this_texture = np.hstack( (np.mean(this_frame, axis=0),np.std(this_frame, axis=0)) )
        tx.append(this_texture)
   
    tx = np.array(tx)
    texture_vector = np.hstack( (tx.mean(axis=0), tx.std(axis=0)) )
    return texture_vector

def audio_to_vec_handcrafted_features(y):
    # Extract features
    mfcc_ = librosa.feature.mfcc(y=y).T
    rms_ = librosa.feature.rms(y=y).T
    cent_ = librosa.feature.spectral_centroid(y=y).T
    flat_ = librosa.feature.spectral_flatness(y=y).T
    roll_ = librosa.feature.spectral_rolloff(y=y).T
   
    features = np.hstack ( (mfcc_, rms_, cent_, flat_, roll_) )
    features_d = np.diff(features)
    features_dd = np.diff(features_d)
   
    texture_vector = np.hstack( (textures(features, 30, 15), textures(features_d, 30, 15), textures(features_dd, 30, 15)) )
    return texture_vector
   
# y, sr = librosa.load(fname, sr=22050)
# feats = audio_to_vec_handcrafted_features(y)

Creating an empty DataFrame

In [3]:
import pandas as pd
column_names = [f'feature_{i+1}' for i in range(276)]
column_names += ['track_id']
df = pd.DataFrame(columns=column_names)

Path for the folder contaning the audios files 

In [4]:
import os
from tqdm import tqdm
import librosa
import pandas as pd

# Supondo que df já foi definido anteriormente
# df = pd.DataFrame(columns=['Col1', 'Col2', ..., 'TrackID'])

directory_path = "../data/fma_large/"
arquivos_problematicos = []  # Lista para guardar os nomes dos arquivos problemáticos

for root, dirs, files in tqdm(os.walk(directory_path)):
    for filename in files:
        if filename.endswith(".mp3"):
            try:
                file_path = os.path.join(root, filename)
                track_id = filename.split('.')[0].lstrip("0")
                y, sr = librosa.load(file_path, sr=22050)
                feats = audio_to_vec_handcrafted_features(y)
                new_row_data = list(feats) + [track_id]
                df.loc[len(df)] = new_row_data
            except Exception as e:  # Captura qualquer exceção que ocorra
                arquivos_problematicos.append(filename)  # Adiciona o nome do arquivo à lista

# Aqui, você pode lidar com os arquivos_problematicos como desejar
print("Arquivos problemáticos:", arquivos_problematicos)


7it [01:59, 14.57s/it]/var/folders/8n/d_gn0k3x4k98s_41gm5jms180000gn/T/ipykernel_26153/932598693.py:18: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=22050)
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/Users/tomasalessi/Workspace/Acuracy-TrainSize/env/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
8it [02:31, 20.21s/it]/var/folders/8n/d_gn0k3x4k98s_41gm5jms180000gn/T/ipykernel_26153/932598693.py:18: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=22050)
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/Users/tomasalessi/Workspace/Acuracy-TrainSize/env/lib/python3.11/s

Arquivos problemáticos: ['105247.mp3', '133297.mp3', '065753.mp3', '098558.mp3', '098571.mp3', '098559.mp3', '098567.mp3', '098560.mp3', '099134.mp3', '001486.mp3', '127336.mp3', '143992.mp3', '126981.mp3', '108925.mp3', '080391.mp3', '005574.mp3']


In [5]:
df.head(277)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,track_id
0,-287.659267,136.153265,-1.044785,46.998524,-4.480115,24.553571,-2.639639,13.259387,7.618672,-2.094758,...,1.933978,1.807735,1.495546,1.251119,1.494395,1.862157,214.978746,428.135264,604.615394,135054
1,-179.123534,112.503843,32.167569,30.893534,3.766597,6.532825,3.143919,3.558495,-9.535322,10.881085,...,4.859566,3.853364,4.128862,3.573876,5.408616,3.928263,401.518601,802.062530,880.749257,135040
2,-99.555557,99.983701,20.608591,21.318870,15.662856,5.197851,-1.226076,0.869772,-8.267710,-1.382700,...,7.598634,5.926319,4.880134,5.931302,7.048175,5.393518,517.643590,1036.383325,1104.245792,135336
3,-108.875410,87.945286,-9.532563,-6.515012,-5.952966,-21.799541,-21.075214,-8.817229,-11.907607,-2.482180,...,6.971639,4.917969,5.534257,8.000605,6.076923,4.345370,497.781151,995.472194,1187.864241,135337
4,-150.223013,99.786985,18.814489,52.378629,17.220552,15.986250,3.212827,1.292879,-12.196205,2.725307,...,5.322011,5.433737,3.980877,5.231651,4.868610,4.262915,401.210943,800.559865,985.653409,135041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,-112.385430,128.331763,-20.412854,31.412043,-0.914894,4.678568,-2.845123,-2.533816,0.290186,-2.310303,...,4.172034,3.802244,2.817600,2.087828,1.805270,1.647323,183.176432,366.199153,663.616736,132044
273,27.035847,61.169455,18.694274,18.136130,10.839069,15.162404,2.626775,9.002828,3.238347,8.844626,...,1.602043,1.872165,2.244891,6.031155,5.851384,3.594959,259.149388,518.144081,601.027942,104276
274,-69.785510,136.562589,41.041460,-5.747020,25.942335,11.383995,0.724751,4.765990,-5.243053,0.428626,...,1.332550,1.837045,1.287269,1.216893,1.395965,1.237638,97.452190,194.272460,272.229753,104262
275,-101.858549,93.262121,10.462528,35.467260,1.539919,21.046988,-3.649734,4.261649,-4.702494,-4.175063,...,2.129587,2.736885,2.330997,2.253795,2.479327,2.600599,378.633136,756.172166,1127.144387,104074


In [6]:
df.to_csv('FMA_large_handcrafted_features.csv', index=False)

In [ ]:
with open('my_file.txt', 'w') as file:
    # Write each item on a new line
    for item in arquivos_problematicos:
        file.write(f"{item}\n")